# Практическое задание #6
Использование данных о выживаемости на Титанике для построения модели бинарной классификации для признака Survived.

# Загрузка и предобрабока данных

## Загрузка данных

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import f1_score, fbeta_score

In [52]:
import pandas as pd
df = pd.read_csv('train.csv')

In [53]:
survival_count = df.Survived.value_counts() 
survival_percentage = (survival_count / 891) * 100
print(survival_percentage)

Survived
0    61.616162
1    38.383838
Name: count, dtype: float64


61% пассажиров не выжили, а 31% выжили => у нас есть дисбаланс классов

## Разбираемся с пропусками

In [54]:
# посмотреть кол-во пропусков по всем переменным
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [55]:
# заменим пропуски возраста средним значением:
imputer = SimpleImputer(strategy='mean')
df['Age'] = imputer.fit_transform(df[['Age']])

## Преобразование категориальных признаков

у нас в данных есть два категориальных признака: 
* Sex - пол (male/female)
* Pclass - класс пассажира (1/2/3)

### Преобразоавние переменной "Sex"

In [56]:
ohe = OneHotEncoder()
res = ohe.fit_transform(df['Sex'].to_numpy().reshape(-1, 1))
ohe_df = pd.DataFrame(res.toarray(), columns=ohe.get_feature_names_out())
ohe_df.head()

,x0_female,x0_male
0,0.0,1.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [57]:
# удаляем один солбец
res = pd.get_dummies(df['Sex'], prefix='sex', drop_first=True)
res.head()

,sex_male
0,True
1,False
2,False
3,False
4,True


In [58]:
# удаляем переменную "Sex" и прцепляем "sex_male"
df = pd.concat([df, res], axis=1).drop(columns='Sex')
df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,sex_male
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,False
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,False
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,True


### Преобразоавние переменной "Pclass"

In [59]:
ohe = OneHotEncoder()
res2 = ohe.fit_transform(df['Pclass'].to_numpy().reshape(-1, 1))
ohe_df = pd.DataFrame(res2.toarray(), columns=ohe.get_feature_names_out())
ohe_df.head()

,x0_1,x0_2,x0_3
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [60]:
# удаляем один солбец
res2 = pd.get_dummies(df['Pclass'], prefix='Pclass', drop_first=True)
res2.head()

,Pclass_2,Pclass_3
0,False,True
1,False,False
2,False,True
3,False,False
4,False,True


In [61]:
# удаляем переменную "Pclass" и прцеляем "Pclass_2" и "Pclass_3"
df = pd.concat([df, res2], axis=1).drop(columns='Pclass')
df.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,sex_male,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,True,False,True
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,False,False,False
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,False,True
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,False,False,False
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,True,False,True


# Строим модель

## Выбор признаков и деление на обучающую и тестовую выборку

In [62]:
# Выбор признаков и целевой переменной
# 1 - survived; 0 - died

features = df[['Pclass_2', 'Pclass_3', 'sex_male', 'Age', 'Fare']]
target = df['Survived']

In [63]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=2023)
X_train

,Pclass_2,Pclass_3,sex_male,Age,Fare
38,False,True,False,18.000000,18.0000
269,False,False,False,35.000000,135.6333
141,False,True,False,22.000000,7.7500
490,False,True,True,29.699118,19.9667
514,False,True,True,24.000000,7.4958
...,...,...,...,...,...
732,True,False,True,29.699118,0.0000
695,True,False,True,52.000000,13.5000
454,False,True,True,29.699118,8.0500
537,False,False,False,30.000000,106.4250


##  Константный бейслайн

In [64]:
# DummyClassifier: классификатор, который всегда предсказывает наиболее часто встречающееся значение из тренировочных данных (стратегия most_frequent).
dummy_clf = DummyClassifier(strategy="most_frequent")

# Обучение "пустой" модели. Модель просто смотрит, какое значение чаще всего встречается в метках и всегда его предсказывает.
dummy_clf.fit(X_train, y_train);

# Получаем предсказание от глупого кллассификатора (модель делает предсказания на тестовых данных)
y_pred_dummy = dummy_clf.predict(X_test)

## Построение модели логистической регрессии

In [65]:
# масштабируем данные
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns
)

In [66]:
# LogisticRegression
log_reg = LogisticRegression()

# Обучаем модель на тренировочных данных
log_reg.fit(X_train_scaled, y_train);

# Делаем предсказания на тестовой выборке
y_preds_logreg = log_reg.predict(X_test_scaled)

## Оценка качества модели

Такая метрика, как Accuracy, не совсем нам подходит в данном случае, так как 60% всех наблюдений — это невыжившие пассажиры, т.е. в нашей выборке есть дисбаланс классов. Это создает риск того, что модель может достигать высокой точности, просто предсказывая "не выживет" для большинства случаев.

Соответственно можно подробнее изучить другие метрики: точность (precision), полнота (recall) и F1 score. Тем не менее, в качестве основного ориентира предлагаю использовать F1 score, так как эта метрика представляет собой гармоническое среднее между точностью (precision) и полнотой (recall), что позволяет учесть обе метрики в одном значении и педоставить более полное предсавление о производительности модели.

### Precision (Точность) & Recall (Полнота)

In [67]:
# строим матрицу ошибок для dummy классификатора:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
confusion_matrix(y_test, y_pred_dummy)

array([[162,   0],
       [106,   0]])

из аутпута видно, что наша dummy модель вообще не угадывает тех, кто выжил, 162 человека она угадывает верно, что они погибли, а в случае 106 человек модель ошибается

In [68]:
# строим матрицу ошибок для логистической регрессии:
confusion_matrix(y_test, y_preds_logreg)

array([[136,  26],
       [ 30,  76]])

из аутпута видно, что лог. модель правильно относит 136 человек к классу 0 (умер), 76 человек правильно относит к классу 1 (выжил), но модель сделала 26 ошибок ложно-позитивных (26 человек отнесла к выжившим, хотя они погибли) и 30 ошибок ложно-негативных (30 человек отнесла к погибшим, хотя они выжили)

In [69]:
print(f'Precision for Dummy Classificator: {precision_score(y_test, y_pred_dummy, zero_division=0)}')
print(f'Precision for Logistic regression: {precision_score(y_test, y_preds_logreg, zero_division=0)}')

Precision for Dummy Classificator: 0.0
Precision for Logistic regression: 0.7450980392156863


74% из всех пассажиров, которых модель определила как выживших, действительно выжили, а 26% — ошибочно были предсказаны как выжившие, но на самом деле погибли.

In [70]:
print(f'Recall for Dummy Classificator: {recall_score(y_test, y_pred_dummy)}')
print(f'Recall for Logistic regression: {recall_score(y_test, y_preds_logreg)}')

Recall for Dummy Classificator: 0.0
Recall for Logistic regression: 0.7169811320754716


модель нашла 71% всех выживших пассажиров, но 29% выживших были ошибочно классифицированы как погибшие

### F1-score

In [71]:
print(f'F1-score for Dummy Classificator: {f1_score(y_test, y_pred_dummy)}')
print(f'F1-score for Logistic regression: {f1_score(y_test, y_preds_logreg)}')

F1-score for Dummy Classificator: 0.0
F1-score for Logistic regression: 0.7307692307692308


Это значение F1 score ближе к 1, чем к 0, что является положительным знаком. В целом, модель способна правильно классифицировать значительное количество выживших и погибших, но все же имеет ложные срабатывания  требует улучшения.